# Biblioteka Bokeh

Bokeh to biblioteka wyróżniająca się wysokim poziomem interaktywności. Umożliwia dostosowywanie wizualizacji w czasie rzeczywistym  dla użytkowników, którzy nie mają styczności z kodem.

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file, show
# plus inne dodatkowe potrzebne do przygotowywanych wizualizacji

In [ ]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns

--------------------

##### ⭐ Zadanie 1: 

Przygotuj wykres punktowy (`circle`) opierając się na danych, które sam wybierzesz w sensowny sposób. W rozwiązaniu zdefiniuj własny pasek narzędzi składający się z narzędzi:

- `LassoSelectTool` i `xPanTool` z kategorii narzędzi *Gestures* (*Pan/Drag tools*),
- `PolySelectTool` z kategorii narzędzi *Gestures* (*Click/Tap tools*),
- `WheelZoomTool` z kategorii narzędzi *Gestures* (*Scroll/Pinch tools*),
- `UndoTool`, `RedoTool`, `ResetTool` i `SaveTool` z kategorii narzędzi *Actions*,
- `HoverTool` z etykietami danych z kategorii narzędzi *Inspectors*.

Zapoznaj się z parametrem `toolbar_location` i dobierz dla niego nową wartość. Zapoznaj się z metodą `autohide` obiektu `toolbar` i dobierz dla niego taką wartość, żeby pasek narzędzi chował się po zjechaniu kursorem myszy z wizualizacji. Zadbaj o czytelność wykresu (tytuł wykresu i podpisy osi). 

In [ ]:
# tutaj wpisz swoje rozwiązanie

##### ⭐ Zadanie 2:

Przygotuj wykres liniowy (`line`) opierając się na danych, które sam wybierzesz w sensowny sposób. Uwzględnij co najmniej 3 serie danych. Każda seria danych musi być przedstawiona na osobnym podwykresie (`subplot`) jednego obrazu. Zapoznaj się z metodami `column`, `row` i `gridplot` oraz dobierz dla nich właściwą wartość mając na uwadze wizualizację określonej liczby serii danych na osobnych podwykresach. Dodaj powiązane zachowania pomiędzy podwykresami:

- współdzielony ruchomy zakres dla wartości na osi X, także przy poziomym przesuwaniu wykresu,
- współdzielony wskaźnik będący narzędziem `CrosshairTool` widoczny jednocześnie na wszystkich podwykresach.

Zadbaj o czytelność wykresu (tytuł wykresu, podpisy osi i ewentualnie legenda). 

In [ ]:
# tutaj wpisz swoje rozwiązanie

##### ⭐ Zadanie 3:

Przygotuj wykres liniowy (`multi_line`) opierając się na danych, które sam wybierzesz w sensowny sposób. Uwzględnij co najmniej 3 serie danych. Zapoznaj się z parametrem `location` obiektu `legend` i dobierz dla niego nową wartość. Zapoznaj się z parametrem `click_policy` obiektu `legend` i dobierz dla niego taką wartość, żeby po kliknięciu w tytuł danej serii danych w legendzie, jej wizualizacji znikała z wykresu. Zadbaj o czytelność wykresu (tytuł wykresu, podpisy osi i legenda). 

In [ ]:
# tutaj wpisz swoje rozwiązanie

##### ⭐ Zadanie 4:

Przedstaw na dowolnym wykresie dowolne zestawienie danych przygotowane na podstawie pliku `athlete_events.csv` z pierwszego tygodnia. Zadbaj o czytelność wykresu. Do swojej wizualizacji dodaj co najmniej 3 różne widgety, które będą miały na nią wpływ:

- `AutocompleteInput`,
- `Button`,
- `CheckboxButtonGroup`,
- `CheckboxGroup`,
- `ColorPicker`,
- `DataCube`,
- `DataTable`,
- `DatePicker`,
- `DateRangePicker`,
- `MultipleDatePicker`,
- `DatetimePicker`,
- `DatetimeRangePicker`,
- `MultipleDatetimePicker`,
- `TimePicker`,
- `DateRangeSlider`,
- `DateSlider`,
- `DatetimeRangeSlider`,
- `Div`,
- `Dropdown`,
- `FileInput`,
- `MultiChoice`,
- `MultiSelect`,
- `NumericInput`,
- `Paragraph`,
- `PasswordInput`,
- `PreText`,
- `RadioButtonGroup`,
- `RadioGroup`,
- `RangeSlider`,
- `Select`,
- `Slider`,
- `Spinner`,
- `Switch`,
- `Tabs`,
- `TextAreaInput`,
- `TextInput`,
- `Toggle`.

Dodatkowo, zastosuj `HelpButton` z `Tooltip` lub dodaj `Tooltip` do przynajmniej 1 widgetu.

In [ ]:
# tutaj wpisz swoje rozwiązanie